In [ ]:
! pip install transformers datasets torch


In [6]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader
import torch.optim as optim
import torch
import numpy as np

## Making Model and Tokenizer :

In [ ]:

model_name = 'distilbert-base-uncased'  

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = 2)  # for P and N labels you knwo this


## get the IMBD Dataset :

In [ ]:
from datasets import load_dataset
dataset = load_dataset('imdb')
dataset

## Make Dataset and Dataloader :

In [9]:
train_data = dataset['train']
test_data = dataset['test']

train_loader = DataLoader(dataset = train_data, batch_size = 16, shuffle = True)
test_loader = DataLoader(dataset = test_data, batch_size = 16, shuffle = True)




In [10]:
from torch.utils.data import random_split
train_data = dataset['train']
train_size = int(0.8* len(train_data))
test_size = len(train_data)-train_size
train_data, test_data = random_split(train_data, [train_size, test_size])

In [11]:



optimizer = optim.Adam(model.parameters(), lr = 3e-5)
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Training the Model :

In [ ]:
model.to(device)
num_epoch = 3

for epoch in range(num_epoch):
  model.train()

  for batch in train_loader:           # you can say this kind alsoo for batch_idx ,(input,label) in enumerat(train_dataloader):   😂😂🤣
    inputs = batch['text']
    labels = batch['label']
    input_ids = tokenizer(inputs, return_tensors='pt', padding= True, truncation= True).to(device)
    labels = torch.tensor(batch['label']).to(device)

    outputs = model(**input_ids, labels = labels)
    loss = criterion(outputs.logits, labels)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


    #if (batch+1)%200==0:
    for i in range(len(train_loader)):
      if i>= 200:

        print(f"Epoch : {epoch}/{num_epoch}, Loss : {loss.item():.4f}")



## evaluat MOdel :

In [14]:
model.to(device)

n_correct = 0
n_sample = 0

with torch.no_grad():
  model.eval()
  for inputs , labels in test_loader:
    input_ids = tokenizer(inputs, return_tensors= 'pt', padding = True, truncation = True)
    labels = [1 if l == 'pos' else 0 for l in labels]
    labels = torch.tensor(labels)

    inputs = input_ids.to(device)
    labels = labels.to(device)

    outputs = model(**input_ids) # to show the clear number you knw
    _, prediction  = torch.max(outputs.logits, dim = -1)    # this is so good for the _, for the tow value you want to give yo but you have one value it ge error you nwo soo _, for "nadida kerftan "one value in yo give it use for that
    # prediction = torch.softmax(outputs.logits, dim = 1)  # it get the eooro for why it do this with me in this like case with the max it do not give the error 😂🤣😂🤣
    n_sample += labels.size(0)
    n_correct += (prediction == labels).sum().item()  # mean if them correct sum the value of that clear on the correct varrible you know


overal_accuracy = 100.0 * n_correct/n_sample
print(f" Overal Accuraci is : {overal_accuracy:.2f}%")






 Overal Accuraci is : 100.00%


In [ ]:
torch.save(model.state_dict(), 'Sentimet_Analysis')

### Test Model :

In [33]:
import torch
texts = ['I love it !', 'taht is very bad !']
label = ['posetive', 'nigative']

with torch.no_grad():

  label2id = {'nigative' :0, 'posetive':1}
  label = torch.tensor([label2id[l] for l in label])  # not give to the cahged to the tensor with the tiuple you know give  you that to the array thing []

  inputs = tokenizer(text= texts, return_tensors= 'pt', padding = True, truncation= True)  # it return that to the tensor automatically !!!!!

  label = label.to(device)
  inputs = inputs.to(device)

  output = model(**inputs)
  # if you wan to see what class predict use this torch.argmax(output, dim =-1)
  probs = torch.softmax(output.logits,dim =1)  #  logits if remove it get error , the loggits is the number of each class that pred from the model so the
                                               #   softmax it get to % to pred each class you know now that is what and for waht we use herr to give the
                                               #   logits the number not another thing you knwo so it is buldin in the CELoss the softmax

  labels = [model.config.id2label[i] for i in range(model.config.num_labels)]

  for i, p in enumerate(probs):
    print(texts[i] , "-->" .join(f'{label[j]} : {p[j].item()*100.0:.1f}%' for j in range(len(label))))

I love it ! 1 : 52.1%-->0 : 47.9%
taht is very bad ! 1 : 51.3%-->0 : 48.7%
